<a href="https://colab.research.google.com/github/meriambar123/memoire/blob/main/Add_personnlizer_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# download presidio
!pip install presidio_analyzer presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 852.9 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install --upgrade --quiet  langchain langchain-experimental langchain-openai presidio-analyzer presidio-anonymizer spacy Faker faiss-cpu tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/7

In [4]:
gros_text="""Date : 19 octobre 2021
Témoin : John Doe

Bonjour Officier,

Je m'appelle John Doe et le 19 octobre 2021, on m'a volé mon portefeuille dans les environs de Kilmarnock lors d'une excursion à vélo. Ce portefeuille contient des objets qui me sont très importants.

Premièrement, le portefeuille contient ma carte de crédit avec le numéro 4111111111111111, qui est enregistrée sous mon nom et liée à mon compte bancaire, PL61109010140000071219812874.

De plus, le portefeuille contenait un permis de conduire - N° de permis : 999000680 émis à mon nom. Il contient également mon numéro de sécurité sociale, 602-76-4532.

En outre, j'avais ma carte d'identité polonaise, avec le numéro ABC123456.

Je souhaite que ces données soient sécurisées et protégées par tous les moyens possibles. Je crois qu'il a été volé à 9h30.

Si jamais des informations apparaissent concernant mon portefeuille, veuillez me contacter par téléphone au 999-888-7777 ou par e-mail personnel, johndoe@example.com.

Veuillez considérer ces informations comme hautement confidentielles et respecter ma vie privée.

La banque a été informée du vol de la carte de crédit et les actions nécessaires ont été prises de leur côté. Ils seront joignables via leur e-mail officiel, support@bankname.com. Ma représentante là-bas est Victoria Cherry (son téléphone professionnel : 987-654-3210).

Merci pour votre aide,

Hedi Turki
Téléphone : 0654859475
IBAN Français : FR6110901014000007121981874 (RIB français)
Sécurité sociale française : 121212121212
Sécurité sociale maladie : 201069935115371
Numéro de permis français : ABV123156324561
Adresse format français : 43 avenue colonel zebi 93100, Montreuil, Île-de-France

On se retrouve à 9h30

J'avais ma carte d'identité polonaise là-bas, avec le numéro ABC123456.
J'avais ma carte d'identité polonaise là-bas, avec le numéro ABC123456.
Adresse IP : 52.222.158.75

Voici mon ID polonais : ABC123456 Mon numéro de sécurité sociale : 201069935115371, sécurité sociale 112233 de Victor Hugo, mon passeport : 12AB12359, oui oui je m'excuse
John Doe"""

gros_text_anglais = """Date: October 19, 2021
 Witness: John Doe
 Subject: Testimony Regarding the Loss of Wallet

 Testimony Content:

 Hello Officer,

 My name is John Doe and on October 19, 2021, my wallet was stolen in the vicinity of Kilmarnock during a bike trip. This wallet contains some very important things to me.

 Firstly, the wallet contains my credit card with number 4111 1111 1111 1111, which is registered under my name and linked to my bank account, PL61109010140000071219812874.

 Additionally, the wallet had a driver's license - DL No: 999000680 issued to my name. It also houses my Social Security Number, 602-76-4532.

 What's more, I had my polish identity card there, with the number ABC123456.

 I would like this data to be secured and protected in all possible ways. I believe It was stolen at 9:30 AM.

 In case any information arises regarding my wallet, please reach out to me on my phone number, 999-888-7777, or through my personal email, johndoe@example.com.

 Please consider this information to be highly confidential and respect my privacy.

 The bank has been informed about the stolen credit card and necessary actions have been taken from their end. They will be reachable at their official email, support@bankname.com.
 My representative there is Victoria Cherry (her business phone: 987-654-3210).

 Thank you for your assistance,

 John Doe"""

In [5]:
from presidio_analyzer import Pattern, PatternRecognizer
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngineProvider

LANGUAGES_CONFIG_FILE = "/content/languages-config.yml"

# Create NLP engine based on configuration file
provider = NlpEngineProvider(conf_file=LANGUAGES_CONFIG_FILE)
nlp_engine_with_french = provider.create_engine()

context_nir=["securite","sociale", "maladie"]
NIR_pattern = Pattern(
    name="nir_pattern",
    regex="[1-478][0-9]{2}(0[1-9]|1[0-2]|62|63)(2[ABab]|[0-9]{2})(00[1-9]|0[1-9][0-9]|[1-8][0-9]{2}|9[0-8][0-9]|990)(00[1-9]|0[1-9][0-9]|[1-9][0-9]{2})(0[1-9]|[1-8][0-9]|9[0-7])",
    score=1)
nir_recognizer = PatternRecognizer(supported_entity="NIR", patterns=[NIR_pattern],context=context_nir, supported_language="fr")
polish_id_pattern = Pattern(
    name="polish_id_pattern",
    regex="[A-Z]{3}\d{6}",
    score=1,
)
polish_id_recognizer = PatternRecognizer(
    supported_entity="POLISH_ID", patterns=[polish_id_pattern], supported_language="fr"
)
context_passeport=["voyage","document", "passeport"]
passeport_pattern = Pattern(
    name="passeport_pattern",
    regex="[0-9]{2}[A-z]{2}[0-9]{5}",
    score=1)
passeport_recognizer = PatternRecognizer(supported_entity="PASSEPORT", patterns=[passeport_pattern],context=context_passeport, supported_language="fr")

context_rib=["banque","IBAN", "compte"]
rib_pattern = Pattern(
    name="rib_pattern",
    regex="FR\d{12}[0-9A-Z]{11}\d{2}",
    score=1)
rib_recognizer = PatternRecognizer(supported_entity="RIB", patterns=[rib_pattern],context=context_rib, supported_language="fr")



In [42]:
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
registry = RecognizerRegistry(supported_languages=["en","fr"])
registry.load_predefined_recognizers()
#registry.remove_recognizer("UsBankRecognizer")
#registry.remove_recognizer("UsLicenseRecognizer")

# Add the recognizer to the existing list of recognizers
registry.add_recognizer(nir_recognizer)
registry.add_recognizer(polish_id_recognizer)
registry.add_recognizer(passeport_recognizer)
registry.add_recognizer(rib_recognizer)

# Set up analyzer with our updated recognizer registry
# analyzer = AnalyzerEngine(registry=registry)

analyzer = AnalyzerEngine(
    registry=registry,
    supported_languages=["en","fr"],
    nlp_engine=nlp_engine_with_french)
# Run with input text
text1="Voici mon id polonais: ABC123456 Mon numéro de securite sociale 201069935115371, securite sociale 112233 de Victor Hugo mon passeport : 12AB12359 oui oui je m'excuse"
text="Voici mon id polonais: ABC123456"
text2=" my bank account, GB74HAVG65414586850142"

results = analyzer.analyze(text=gros_text, language="fr")
print(results)

[type: POLISH_ID, start: 687, end: 696, score: 1, type: PERSON, start: 33, end: 110, score: 0.85, type: LOCATION, start: 446, end: 474, score: 0.85, type: LOCATION, start: 478, end: 490, score: 0.85, type: PERSON, start: 863, end: 904, score: 0.85, type: LOCATION, start: 1216, end: 1236, score: 0.85, type: PERSON, start: 1353, end: 1361, score: 0.85]


In [8]:
import re


def print_colored_pii(string):
    colored_string = re.sub(
        r"(<[^>]*>)", lambda m: "\033[31m" + m.group(1) + "\033[0m", string
    )
    print(colored_string)

In [9]:
from faker import Faker

fake = Faker()


def fake_polish_id(_=None):
    return fake.bothify(text="???######").upper()


fake_polish_id()

'JPC166500'

In [10]:
from presidio_anonymizer.entities import OperatorConfig

new_operators = {
    "POLISH_ID": OperatorConfig("custom", {"lambda": fake_polish_id})
}



In [9]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer
from presidio_anonymizer import AnonymizerEngine

nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "en", "model_name": "en_core_web_md"},
        {"lang_code": "fr", "model_name": "fr_core_news_lg"},
    ],
}

anonymizer = PresidioReversibleAnonymizer(add_default_faker_operators=False, languages_config=nlp_config)
# engine = AnonymizerEngine()

# result1 = engine.anonymize(
#     text=gros_text,
#     analyzer_results=results
# )

# print(result1)
# print(type(result1))

anonymizer.add_recognizer(polish_id_recognizer)
anonymizer.add_recognizer(nir_recognizer)
anonymizer.add_recognizer(rib_recognizer)
anonymizer.add_recognizer(passeport_recognizer)
print_colored_pii(anonymizer.anonymize(gros_text, language="fr"))

Date : 19 octobre 2021
<PERSON> : <PERSON_2>

Bonjour Officier,

Je m'appelle <PERSON_2> et le 19 octobre 2021, on m'a volé mon portefeuille dans les environs de <LOCATION> lors d'une excursion à vélo. Ce portefeuille contient des objets qui me sont très importants.

Premièrement, le portefeuille contient ma carte de crédit avec le numéro <NIR>1, qui est enregistrée sous mon nom et liée à mon compte bancaire, <IBAN_CODE>.

De plus, le portefeuille contenait un permis de conduire - N° de permis : 999000680 émis à mon nom. Il contient également mon numéro de sécurité sociale, 602-76-4532.

En outre, j'avais ma carte d'identité polonaise, avec le numéro <POLISH_ID>.

Je souhaite que ces données soient sécurisées et protégées par tous les moyens possibles. Je crois qu'il a été volé à 9h30.

Si jamais des informations apparaissent concernant mon portefeuille, veuillez me contacter par téléphone au <PHONE_NUMBER> ou par e-mail personnel, <EMAIL_ADDRESS>.

Veuillez considérer ces informations

text: Date : 19 octobre 2021
<PERSON> : <PERSON>

Bonjour Officier,

Je m'appelle <PERSON> et le 19 octobre 2021, on m'a volé mon portefeuille dans les environs de <LOCATION> lors d'une excursion à vélo. Ce portefeuille contient des objets qui me sont très importants.

Premièrement, le portefeuille contient ma carte de crédit avec le numéro <NIR>1, qui est enregistrée sous mon nom et liée à mon compte bancaire, <LOCATION>.

De plus, le portefeuille contenait un permis de conduire - N° de permis : 999000680 émis à mon nom. Il contient également mon numéro de sécurité sociale, 602-76-4532.

En outre, j'avais ma carte d'identité polonaise, avec le numéro <POLISH_ID>.

Je souhaite que ces données soient sécurisées et protégées par tous les moyens possibles. Je crois qu'il a été volé à 9h30.

Si jamais des informations apparaissent concernant mon portefeuille, veuillez me contacter par téléphone au 999-888-7777 ou par e-mail personnel, johndoe@example.com.

Veuillez considérer ces informations comme hautement confidentielles et respecter ma vie privée.

La banque a été informée du vol de la carte de crédit et les actions nécessaires ont été prises de leur côté. Ils seront joignables via leur e-mail officiel, <LOCATION>. Ma représentante là-bas est <PERSON> (son téléphone professionnel : 987-654-3210).

Merci pour votre aide,

<PERSON>
Téléphone : 0654859475
IBAN <LOCATION> : <RIB> (RIB français)
Sécurité sociale française : 121212121212
Sécurité sociale maladie : <NIR>
Numéro de permis français : <PERSON> format français : 43 avenue colonel zebi 93100, <LOCATION>, <LOCATION>

On se retrouve à 9h30

J'avais ma carte d'identité polonaise là-bas, avec le numéro <POLISH_ID>.
J'avais ma carte d'identité polonaise là-bas, avec le numéro <POLISH_ID>.
Adresse IP : 52.222.158.75

Voici mon ID polonais : <POLISH_ID> Mon numéro de sécurité sociale : <NIR>, sécurité sociale 112233 de <PERSON>, mon passeport : <PASSEPORT>, oui oui je m'excuse
<PERSON>
items:
[
    {'start': 1953, 'end': 1961, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 1920, 'end': 1931, 'entity_type': 'PASSEPORT', 'text': '<PASSEPORT>', 'operator': 'replace'},
    {'start': 1894, 'end': 1902, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 1860, 'end': 1865, 'entity_type': 'NIR', 'text': '<NIR>', 'operator': 'replace'},
    {'start': 1815, 'end': 1826, 'entity_type': 'POLISH_ID', 'text': '<POLISH_ID>', 'operator': 'replace'},
    {'start': 1750, 'end': 1761, 'entity_type': 'POLISH_ID', 'text': '<POLISH_ID>', 'operator': 'replace'},
    {'start': 1676, 'end': 1687, 'entity_type': 'POLISH_ID', 'text': '<POLISH_ID>', 'operator': 'replace'},
    {'start': 1580, 'end': 1590, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 1568, 'end': 1578, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 1511, 'end': 1519, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 1477, 'end': 1482, 'entity_type': 'NIR', 'text': '<NIR>', 'operator': 'replace'},
    {'start': 1387, 'end': 1392, 'entity_type': 'RIB', 'text': '<RIB>', 'operator': 'replace'},
    {'start': 1374, 'end': 1384, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 1337, 'end': 1345, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 1257, 'end': 1265, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 1217, 'end': 1227, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 654, 'end': 665, 'entity_type': 'POLISH_ID', 'text': '<POLISH_ID>', 'operator': 'replace'},
    {'start': 409, 'end': 419, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 337, 'end': 342, 'entity_type': 'NIR', 'text': '<NIR>', 'operator': 'replace'},
    {'start': 158, 'end': 168, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 76, 'end': 84, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 34, 'end': 42, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'},
    {'start': 23, 'end': 31, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'}
]

<class 'presidio_anonymizer.entities.engine.result.engine_result.EngineResult'>

In [48]:
print(anonymizer.deanonymizer_mapping)


{}


Utilisation PresidioReveribleAnonymizer avec langue française configurée.

Date : 19 octobre 2021
<PERSON> : <PERSON_2>

Bonjour Officier,

Je m'appelle <PERSON_2> et le 19 octobre 2021, on m'a volé mon portefeuille dans les environs de <LOCATION> lors d'une excursion à vélo. Ce portefeuille contient des objets qui me sont très importants.

Premièrement, le portefeuille contient ma carte de crédit avec le numéro <NIR>1, qui est enregistrée sous mon nom et liée à mon compte bancaire, <IBAN_CODE>.

De plus, le portefeuille contenait un permis de conduire - N° de permis : 999000680 émis à mon nom. Il contient également mon numéro de sécurité sociale, 602-76-4532.

En outre, j'avais ma carte d'identité polonaise, avec le numéro <POLISH_ID>.

Je souhaite que ces données soient sécurisées et protégées par tous les moyens possibles. Je crois qu'il a été volé à 9h30.

Si jamais des informations apparaissent concernant mon portefeuille, veuillez me contacter par téléphone au <PHONE_NUMBER> ou par e-mail personnel, <EMAIL_ADDRESS>.

Veuillez considérer ces informations comme hautement confidentielles et respecter ma vie privée.

La banque a été informée du vol de la carte de crédit et les actions nécessaires ont été prises de leur côté. Ils seront joignables via leur e-mail officiel, <EMAIL_ADDRESS_2>. Ma représentante là-bas est <PERSON_3> (son téléphone professionnel : <PHONE_NUMBER_2>).

Merci pour votre aide,

<PERSON_4>
Téléphone : <PHONE_NUMBER_3>
IBAN <LOCATION_2> : FR6110901014000007121981874 (RIB français)
Sécurité sociale française : 121212121212
Sécurité sociale maladie : <NIR_2>
Numéro de permis français : <PERSON_5> format français : 43 avenue colonel zebi 93100, <LOCATION_3>, <LOCATION_4>

On se retrouve à 9h30

J'avais ma carte d'identité polonaise là-bas, avec le numéro <POLISH_ID>.
J'avais ma carte d'identité polonaise là-bas, avec le numéro <POLISH_ID>.
Adresse IP : <IP_ADDRESS>

Voici mon ID polonais : <POLISH_ID> Mon numéro de sécurité sociale : <NIR_2>, sécurité sociale 112233 de <PERSON_6>, mon passeport : <PASSEPORT>, oui oui je m'excuse
<PERSON_2>